In [3]:
import numpy as np
import os
import pandas as pd
from scipy.fft import fft, fftfreq

# Path to the folder containing .npy files
train_data_path = "/content/"

# List all .npy files in the folder
file_list = [f for f in os.listdir(train_data_path) if f.endswith('.npy')]

# Function to extract FFT coefficients
def extract_fft_coefficients(eeg_data, sampling_rate=500):
    coefficients = []
    n = eeg_data.shape[1]  # Number of time points (500)
    frequencies = fftfreq(n, d=1/sampling_rate)  # Frequency bins

    for channel in range(eeg_data.shape[0]):  # Loop through 19 channels
        signal = eeg_data[channel, :]
        fft_values = fft(signal)  # Get the FFT coefficients (complex values)

        # Take only the positive frequencies (one-sided spectrum)
        positive_freqs = frequencies[:n//2]
        positive_fft_values = np.abs(fft_values[:n//2])  # Use magnitude of FFT coefficients

        # Store the frequencies and corresponding FFT magnitudes
        for freq, fft_val in zip(positive_freqs, positive_fft_values):
            coefficients.append({
                "channel": channel + 1,  # Channel number (1-based index)
                "frequency": freq,
                "fft_magnitude": fft_val
            })
    return coefficients

# List to hold all the data for CSV saving
all_fft_data = []

# Loop through all .npy files in the folder
for file_name in file_list:
    file_path = os.path.join(train_data_path, file_name)
    eeg_data = np.load(file_path)  # Shape: (19, 500)

    # Extract Fourier Transform coefficients (FFT)
    fft_coefficients = extract_fft_coefficients(eeg_data)

    # Add file name to each entry (for reference)
    for entry in fft_coefficients:
        entry["file_name"] = file_name

    # Add the results to the overall list
    all_fft_data.extend(fft_coefficients)

# Convert the data to a pandas DataFrame
df = pd.DataFrame(all_fft_data)

# Save to CSV
output_file = "/content/Out/complex_fft_feature_results.csv"
df.to_csv(output_file, index=False)

print(f"FFT coefficients saved to {output_file}")


FFT coefficients saved to /content/Out/complex_fft_feature_results.csv
